# Extracción de información

* En este taller veremos cómo podemos extraer y analizar el texto a partir de documentos en formato pdf.
* Empezaremos extrayendo el texto de documentos pdf normales y luego de documentos pdf creados a partir de documentos escaneados.
* Guardaremos el texto en un archivo csv para su posterior procesamiento.
* Usaremos herramientas que tengan licencia de software libre y que además permitan automatizar tanto como sea posible el proceso.

# Documentos pdf

* Para este ejemplo usaremos los documentos pdf de los discursos de los miembros de la Junta de Gobierno del Banco de México. Los documentos se encuentran en la página del Banco: https://www.banxico.org.mx/publicaciones-y-prensa/discursos/discursos-junta-gobierno-pala.html

* Estos documentos son pdf "normales", es decir, podemos abrirlos en un visor de pdfs y seleccionar y copiar el contenido. En este caso extraer el texto es una tarea relativamente fácil.

## Librerías

In [1]:
import requests
from bs4 import BeautifulSoup
import os
import time
import PyPDF2
import pandas as pd
import sys

print(requests.__name__, requests.__version__)
print(PyPDF2.__name__, PyPDF2.__version__)
print(pd.__name__, pd.__version__)

requests 2.22.0
PyPDF2 1.26.0
pandas 0.25.3


## Descarga documentos

* Lo primero que necesitamos hacer es descargar los archivos de la página web. En este caso automatizamos la tarea usando la librería [`requests`](https://2.python-requests.org//es/latest/user/quickstart.html) para hacer las peticiones al servidor de la página donde están todos los discursos.
* Usamos la librería [`BeautifulSoup`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) para "parsear" la página web y obtener el enlace a la descarga de cada archivo individual. Una vez tenemos estos enlaces obtenemos cada archivo y los guardamos en un folder de nuestra computadora.
* Excluimos los documentos que están en inglés.

In [2]:
dir_discurso = 'docs/discursos'
url_base = 'https://www.banxico.org.mx'
url_discursos = url_base + '/publicaciones-y-prensa/discursos/discursos-junta-gobierno-pala.html'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'}
resp = requests.get(url_discursos, headers=headers)

soup = BeautifulSoup(resp.content, 'lxml')
documentos = [(tag.select_one('td[class="bmtextview"]').text, tag.select_one('a')['href']) for tag in soup.select('tr')]

for doc in documentos:
    if 'disponible sólo en inglés' in doc[0]:
        continue
    pdfname = f'{dir_discurso}/{doc[1].split("/")[-1]}'
    if not os.path.exists(pdfname):
        time.sleep(1)
        resp = requests.get(url_base+doc[1], headers=headers)
        with open(pdfname, 'wb') as file:
            file.write(resp.content)

## Extrae la información de los pdfs

* Una vez tenemos los archivos descargados vamos a extraer el texto en cada uno de ellos.
* En Python hay muchas librerías que pueden leer pdfs para extraer sus contenido o los metadatos. En este taller usaremos [`PyPDF2`](https://pythonhosted.org/PyPDF2/) porque es una librería hecha enteramente en Python y por tanto no necesitamos instalar otras dependencias.
* El texto que extraemos lo guardamos en una lista que después convertiremos en un DataFrame de [pandas](https://pandas.pydata.org/). Este lo guardamos como un archivo csv.


In [3]:
# Ejemplo de como extraer el texto de un pdf
with open(pdfname, 'rb') as pdf:
    red = PyPDF2.PdfFileReader(pdf)
    cont = [p.extractText() for p in red.pages]
print(cont)

['PALABRAS DEL DR. GUILLERMO ORTIZ, GOBERNADOR DEL BANCO DE MÉXICO,  EN OCASIÓN DE LA SEXAGÉSIMA NOVENA CONVENCIÓN BANCARIA Acapulco, Gro., a 24 de marzo de 2006  Lic. Francisco Gil Díaz Secretario de Hacienda y Crédito Público  Ing. Marcos Martínez Gavica Presidente de la Asociación de Bancos de México  Honorables miembros del Presidium Distinguidos invitados especiales Señoras y Señores,  Me es muy grato participar una vez más en la Convención Bancaria.  En esta edición, aportaré el producto de algunas reflexiones sobre los objetivos que tiene señalados en su ley el Banco de México, haciendo hincapié en una tesis fundamental: que la finalidad ulterior de esos objetivos es tener una economía con capacidad más elevada de crecimiento que contribuya a un mayor bienestar para todos los mexicanos.   Desde su establecimiento, en 1925, el Banco ha procurado el logro de una economía más estable, de un sistema financiero más desarrollado y de un mejor sistema de pagos.  En sus primeras etapas,

In [3]:
#Extrayendo el contendio de todos los pdfs en el folder docs/discursos
texto_discursos = []
for doc in documentos:
    if 'disponible sólo en inglés' in doc[0]:
        continue

    pdfname = f'{dir_discurso}/{doc[1].split("/")[-1]}'
    try:
        with open(pdfname, 'rb') as pdf:
            red = PyPDF2.PdfFileReader(pdf)
            cont = [p.extractText() for p in red.pages]
        entrada =  {'archivo': pdfname, 'titulo': doc[0],'texto': ' '.join(' '.join(cont).split())}
        texto_discursos.append(entrada)
    except Exception as e:
        print('problema con', pdfname, repr(e))

In [ ]:
# Guardando el contenido a una archivo .csv
df = pd.DataFrame(data=texto_discursos)
df.to_csv('datos/discursos.csv', index=False)
df.head()

# Documentos escaneados

* Muchas veces ocurre que el contenido de los documentos pdf son imágenes escaneadas y por tanto el método anterior para extraer su contenido no funcionará.
* En esos casos necesitamos de herramientas que puedan hacer reconocimiento óptico de caracteres (OCR) sobre las imágenes.
* Una herramienta muy utilizada en estos casos es [Tesseract-OCR](https://github.com/tesseract-ocr/tesseract). Existen varias implementaciones de este programa para Python. Una de ellas es [`pytesseract`](https://github.com/madmaze/pytesseract)
    * Para que funcione, debemos instalar una versión de Tesseract en nuestra computadora. Para el sistema operativo Windows se puede descargar un instalador en este repositorio: https://digi.bib.uni-mannheim.de/tesseract/. Para otros sistemas operativos pueden consultar la [guía de instalación](https://github.com/tesseract-ocr/tesseract/wiki#installation).
    * Luego debemos instalar la librería en Python con `pip install pytesseract`
* Para poder usar Tesseract debemos extraer las imágenes escaneadas dentro del pdf. Para eso necesitamos de la libería [`wand`](http://docs.wand-py.org/en/0.5.7/index.html).
    * Para que Wand funcione debemos instalar el programa `ImageMagick` en nuestra computadora. Para el sistema operativo Windows se puede descargar desde [este repositorio](http://legacy.imagemagick.org/script/download.php#windows) y seguir las instrucciones de la [guía de instalación](http://docs.wand-py.org/en/0.5.7/guide/install.html#).
    * También es necesario instalar el programa GhostScript, el cual se puede descargar en [este repositorio](https://www.ghostscript.com/download/gsdnld.html)
    * Instalar la librería Wand en Python con `pip install Wand`
    * Puede que sea necesario ajustar algunas configuraciones de `ImageMagick` como se explica en [esta página](https://stackoverflow.com/questions/54627549/imagemagick-failedtoexecutecommand-gswin32c-exe)
* En el script debemos especificar las rutas donde se instalaron `Tesseract` y `ImageMagick`
* Una vez instaladas las librerías, el proceso consiste en extraer las imágenes del pdf usando `Wand`, luego aplicar OCR a esas imágenes usando `Tesseract`. Para simplificar el proceso  se crea la función `pdf_to_text(pdf_file)`, donde `pdf_file` es la ruta a un archivo pdf. El resultado de la función es un diccionario que contiene el texto de cada página.
* A continuación probamos el script sobre dos documentos pdf obtenidos de la [Plataforna Nacional de Transparencia](https://www.plataformadetransparencia.org.mx/web/guest/inicio) desde donde se puede descargar documentos públicos que en su mayoría están escaneados.

## Librerías

In [2]:
import os
import sys
import pytesseract
from pytesseract import image_to_string
from wand.image import Image as Img
import PIL
import glob
os.environ['TESSDATA_PREFIX'] = 'C:/Program Files (x86)/Tesseract-OCR/tessdata'
#pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files (x86)/Tesseract-OCR'
#os.environ['MAGICK_HOME'] = 'C:/Program Files/ImageMagick-6.9.10-Q16'

In [1]:
def pdf_to_text(pdf_file: str) -> dict:
    """
    Convierte un pdf escaneado a texto. Retorna un diccionario con el contenido de cada página
    """
    fname = pdf_file.split('.pdf')[0].split('/')[-1]
    folder = f'docs/imgs/{fname}'
    os.makedirs(folder, exist_ok=True)
    with Img(filename=pdf_file, resolution=600) as img:
        img.compression_quality = 99
        img.save(filename=f'{folder}/pagina.jpg')
    
    paginas = glob.glob1(folder, '*.jpg')
    output = dict()
    for pag in paginas:
        try:
            output[pag.replace('.jpg', '')] = pytesseract.image_to_string(PIL.Image.open(f'{folder}/{pag}').convert("RGB"),
                                                                         lang='esp')
        except Exception as e:
            print(f'Error en {pag}', repr(e))
    return output


In [4]:
PDF_file = "docs/pbt_DICTAMEN 03-2019-DIAR-02_201909171659.pdf"
pdf1 = pdf_to_text(PDF_file)

In [5]:
for k, v in pdf1.items():
    print('##########', k,'####################', '\n', v, '\n')

########## pagina-0 #################### 
 Sistema de Servicio Profesional

del Instituto Federal de Telecomunicaciones

 

DICTAMEN

La Direccion General de Gestion de Talento y la Direccidn de Administracidon de
Personal, con fundamento en lo dispuesto por los articulos 11 fracciones I, IV y VI, 18
de las "Disposiciones por las que se establece el Sistema de Servicio Profesional
del Instituto Federal de Telecomunicaciones"; 1, 3, fraccidon VIII, 5, 13 y 91 de los
"Lineamientos Especificos en Materia de Ingreso", y 4, fraccion IX, inciso xxiii, 57 y
58, fraccion I del Estatuto Orgéanico del Instituto Federal de Telecomunicaciones, y

CONSIDERANDO

Primero. Que con fecha 14 de junio de 2019 el Instituto Federal de
Telecomunicaciones (en adelante el Instituto o IFT) publico en el sistema
ProTalentolFT la "CONVOCATORIA PUBLICA Y (en adelante la
Convocatoria) identificada con el numero 03-2019 dirigida al publico en general,
para participar en el procedimiento de reclutamiento y seleccion

In [6]:
PDF_file = "docs/CONT-ARRE-SEME-2019-TEST.pdf"
pdf2 = pdf_to_text(PDF_file)

In [7]:
for k, v in pdf2.items():
    print('##########', k,'####################', '\n', v, '\n')

########## pagina-0 #################### 
 CONTRATO DE ARRENDAMIENTO

CONTRATO DE ARRENDAMIENTO, QUE CELEBRAN, POR UNA PARTE EL C. AMADA RAMOS
OSORIO, EN SU CARACTER DE DUENO, A QUIEN EN LO SUCESIVO SE LE DENOMINARA "EL
ARRENDADOR" Y POR LA OTRA, LA FISCALIA GENERAL DE LA REPUBLICA EN SU
REPRESENTACION EN EL ESTADO DE QUERETARO, REPRESENTADA POR EL LIC. CRISTIAN
PAUL CAMACHO OSNAYA , ENCARGADO DEL DESPACHO DE LA DELGACION ESTATAL, A QUIEN

EN LO SUCESIVO SE LE DENOMINARA "LA ARRENDATARIO" AL TENOR DE LAS SIGUIENTES
DECLARACIONES Y CLAUSULAS.

DE CLARA CIONE S

I.-DECLARA "EL ARRENDADOR".

A. Que es propietario del inmueble objeto del presente contrato, con domicilio en calle
boulevard hidalgo #77 col. Centro C.P 76800, San Juan del Rio Qro.; segiun lo que acredita con la
Escritura Publica Numero 2671 , (dos, seis, siete, uno) pasada ante la Fe del Lic. Francisco Esquivel
Rodriguez, notario interno de la Notaria Publica Numero , de la Ciudad de 20, e inscrita en el Registro
Publico de l

In [30]:
# Guardamos como txt
with open('datos/pdf1.txt', 'w', encoding='utf8') as txtfile:
    for k, v in pdf1.items():
        txtfile.write(v + '\n')
        
with open('datos/pdf2.txt', 'w', encoding='utf8') as txtfile:
    for k, v in pdf2.items():
        txtfile.write(v + '\n')